In [ ]:
"""
Similar to `store-pretrained-model-paths.ipynb`, but also exports hidden states. Use for clustering analysis to compare clusters of hidden states versus expert IDs.
"""
None

In [ ]:
"""
Imports
"""
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.loss.loss_utils import ForCausalLMLoss # Cross-entropy loss that handles label shifting
from datasets import load_dataset
import pandas as pd
import numpy as np
from tqdm import tqdm
from termcolor import colored
import importlib 

from utils.memory import check_memory, clear_all_cuda_memory
from utils.store_topk import convert_topk_to_df
from utils.store_outputs import convert_outputs_to_df, convert_hidden_states_to_df
from utils import pretrained_models

main_device = 'cuda:0'
seed = 1234
clear_all_cuda_memory()
check_memory()

## Load base model

In [ ]:
"""
Load the base tokenizer/model

Architectures supported currently:
- OlMoE architecture, includes OLMoE-1B-7B-0125-Instruct (1B/7B)
- Qwen2MoE architecture, inclues Qwen1.5-MoE-A2.7B-Chat (2.7B/14.3B), Qwen2-57B-A14B (14B/57B)
- Deepseek v2 architecture, includes Deepseek-v2-Lite (2.4B/15.7B), Deepseek-v2 (21B/236B)
- Deepseek v3 architecture, includes Deepseek-v3 (37B/671B), Deepseek-R1 (37B/671B), Moonlight-16B-A3B (3B/16B)
"""
selected_model_index = 0

def get_model(index):
    model = [
        ('allenai/OLMoE-1B-7B-0125-Instruct', 'olmoe', 'olmoe'),
        ('Qwen/Qwen1.5-MoE-A2.7B-Chat', 'qwen1.5moe', 'qwen2moe'),
        ('deepseek-ai/DeepSeek-V2-Lite', 'dsv2', 'dsv2'),
        ('moonshotai/Moonlight-16B-A3B', 'moonlight', 'dsv3')
    ][index]

    return model[0], model[1], model[2]

model_id, model_prefix, model_architecture = get_model(selected_model_index)
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token = False, add_bos_token = False, padding_side = 'left', trust_remote_code = True)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype = torch.bfloat16, trust_remote_code = True).cuda().eval()

In [ ]:
"""
Load reverse-engineered forward pass functions that return topk expert IDs and weights
"""
model_module = importlib.import_module(f"utils.pretrained_models.{model_architecture}")
run_model_return_topk = getattr(model_module, f"run_{model_architecture}_return_topk")

def test_custom_forward_pass(model, pad_token_id):
    inputs = tokenizer(['Hi! I am a dog and I like to bark', 'Vegetables are good for'], return_tensors = 'pt', padding = 'max_length', truncation = True, max_length = 512).to(model.device)
    original_results = model(**inputs)
    custom_results = run_model_return_topk(model, inputs['input_ids'], inputs['attention_mask'], return_hidden_states = True)
    assert torch.equal(original_results.logits, custom_results['logits']), 'Error in custom forward'
    assert len(custom_results['all_topk_experts']) == len(custom_results['all_topk_weights']), 'Length of topk IDs and weights not equal'
    print(f"Length of topk: {len(custom_results['all_topk_experts'])}")
    print(f"Topk size: {custom_results['all_topk_experts'][0].shape}")
    print(f"First token topk IDs: {custom_results['all_topk_experts'][0][1,]}")
    print(f"First token topk weights: {custom_results['all_topk_weights'][0][1,]}")
    loss = ForCausalLMLoss(custom_results['logits'], torch.where(inputs['input_ids'] == pad_token_id, torch.tensor(-100), inputs['input_ids']), model.config.vocab_size).detach().cpu().item()
    print(f"LM loss: {loss}")
    print(f"Hidden states layers (pre-mlp, post-layer): {len(custom_results['all_pre_mlp_hidden_states'])} | {len(custom_results['all_hidden_states'])}")
    print(f"Hidden state size (pre-mlp, post-layer): {(custom_results['all_pre_mlp_hidden_states'][0].shape)} | {(custom_results['all_hidden_states'][0].shape)}")

test_custom_forward_pass(model, tokenizer.pad_token_id)

## Get dataset

In [ ]:
"""
Load dataset
"""
def load_raw_ds():
    ds = load_dataset('HuggingFaceFW/fineweb-edu', 'CC-MAIN-2024-51', split = 'train', streaming = True).shuffle(seed = 123, buffer_size = 100_000)
    ds_iter = iter(ds)

    raw_data = []
    for _ in range(0, 500):
        sample = next(ds_iter, None)
        if sample is None:
            break
        raw_data.append(sample['text'])
    
    return raw_data

raw_data = load_raw_ds()

In [ ]:
""" 
Load dataset into a dataloader. The dataloader returns the original tokens - this is important for BPE tokenizers as otherwise it's difficult to reconstruct the correct string later!
"""
from torch.utils.data import Dataset, DataLoader

class ReconstructableTextDataset(Dataset):

    def __init__(self, text_dataset, tokenizer, max_length):
        """
        Creates a dataset object that also returns a B x N list of the original tokens in the same position as the input ids.

        Params:
            @text_dataset: A list of B samples of text dataset.
            @tokenizer: A HF tokenizer object.
        """
        tokenized = tokenizer(text_dataset, add_special_tokens = False, max_length = max_length, padding = 'max_length', truncation = True, return_offsets_mapping = True, return_tensors = 'pt')

        self.input_ids = tokenized['input_ids']
        self.attention_mask = tokenized['attention_mask']
        self.offset_mapping = tokenized['offset_mapping']
        self.original_tokens = self.get_original_tokens(text_dataset)

    def get_original_tokens(self, text_dataset):
        """
        Return the original tokens associated with each B x N position. This is important for reconstructing the original text when BPE tokenizers are used.
        
        Params:
            @input_ids: A B x N tensor of input ids.
            @offset_mapping: A B x N x 2 tensor of offset mappings. Get from `tokenizer(..., return_offsets_mapping = True)`.

        Returns:
            A list of length B, each with length N, containing the corresponding original tokens corresponding to the token ID at the same position of input_ids.
        """
        all_token_substrings = []
        for i in range(0, self.input_ids.shape[0]):
            token_substrings = []
            for j in range(self.input_ids.shape[1]): 
                start_char, end_char = self.offset_mapping[i][j].tolist()
                if start_char == 0 and end_char == 0: # When pads, offset_mapping might be [0, 0], so let's store an empty string for those positions.
                    token_substrings.append("")
                else:
                    original_substring = text_dataset[i][start_char:end_char]
                    token_substrings.append(original_substring)
            
            all_token_substrings.append(token_substrings)

        return all_token_substrings

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return {'input_ids': self.input_ids[idx], 'attention_mask': self.attention_mask[idx], 'original_tokens': self.original_tokens[idx]}
    
def collate_fn(batch):
    """
    Custom collate function; necessary to return original_tokens in the correct shape 
    """
    input_ids = torch.stack([b['input_ids'] for b in batch], dim = 0)
    attention_mask = torch.stack([b['attention_mask'] for b in batch], dim = 0)        
    original_tokens = [b['original_tokens'] for b in batch]
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'original_tokens': original_tokens}

test_dl = DataLoader(
    ReconstructableTextDataset(raw_data, tokenizer, max_length = 1024),
    batch_size = 8,
    shuffle = False,
    collate_fn = collate_fn
)

## Get expert selections + export

In [ ]:
""" 
Run forward passes + export data

Note the bulk of compute time will be spent on exporting the CSV, not handling the forward passes.
"""

@torch.no_grad()
def run_and_export_topk(model, model_prefix, file_suffix, dl, max_batches = None):
    """
    Run forward passes on a given model ID, return topk df
    """
    b_count = 0
    all_pre_mlp_hidden_states = []
    sample_dfs = []
    topk_dfs = []

    for batch_ix, batch in tqdm(enumerate(dl), total = len(dl)):

        input_ids = batch['input_ids'].to(main_device)
        attention_mask = batch['attention_mask'].to(main_device)
        original_tokens = batch['original_tokens']

        output = run_model_return_topk(model, input_ids, attention_mask, return_hidden_states = True)

        # Check no bugs by validating output/perplexity
        if batch_ix == 0:
            loss = ForCausalLMLoss(output['logits'], torch.where(input_ids == tokenizer.pad_token_id, torch.tensor(-100), input_ids), model.config.vocab_size).detach().cpu().item()
            for i in range(min(2, input_ids.size(0))):
                decoded_input = tokenizer.decode(input_ids[i, :attention_mask[i].sum()], skip_special_tokens = True)
                next_token_id = torch.argmax(output['logits'][i, -1, :]).item()
                print(decoded_input + colored(tokenizer.decode([next_token_id], skip_special_tokens = True), 'green'))
            print(f"PPL:", torch.exp(torch.tensor(loss)).item())
        
        original_tokens_df = pd.DataFrame(
            [(seq_i, tok_i, tok) for seq_i, tokens in enumerate(original_tokens) for tok_i, tok in enumerate(tokens)], 
            columns = ['sequence_ix', 'token_ix', 'token']
        )

        # Create sample (token) level dataframe
        sample_df =\
            convert_outputs_to_df(input_ids, attention_mask, output['logits'])\
            .merge(original_tokens_df, how = 'left', on = ['token_ix', 'sequence_ix'])\
            .assign(batch_ix = batch_ix)

        # # We'll analyze the first 5 layers only for simplicity
        # pre_mlp_hidden_state_df =\
        #     convert_hidden_states_to_df(input_ids, attention_mask, output['all_pre_mlp_hidden_states'][0:5])\
        #     .assign(batch_ix = batch_ix)\
        #     .drop(columns = 'token_id')

        topk_df =\
            convert_topk_to_df(input_ids, attention_mask, output['all_topk_experts'], output['all_topk_weights'])\
            .assign(batch_ix = batch_ix, weight = lambda df: df['weight'])\
            .drop(columns = 'token_id')
        
        sample_dfs.append(sample_df)
        all_pre_mlp_hidden_states.append(output['all_pre_mlp_hidden_states'][0:5])
        topk_dfs.append(topk_df)
        # sample_df.to_csv(f'data/hidden-states/{model_prefix}-{file_suffix}-samples.csv', mode = 'w' if batch_ix == 0 else 'a', index = False, header = (batch_ix == 0))
        # all_pre_mlp_hidden_states.append(output['all_pre_mlp_hidden_states'][0:5])
        # pre_mlp_hidden_state_df.to_csv(f'data/hidden-states/{model_prefix}-{file_suffix}-pre-mlp-hidden-states.csv', mode = 'w' if batch_ix == 0 else 'a', index = False, header = (batch_ix == 0))
        # topk_df.to_csv(f'data/hidden-states/{model_prefix}-{file_suffix}-routes.csv', mode = 'w' if batch_ix == 0 else 'a', index = False, header = (batch_ix == 0))
        # topk_df[topk_df['topk_ix'] == 1].to_csv(f'data/hidden-states/{model_prefix}-{file_suffix}-routes-top1.csv', mode = 'w' if batch_ix == 0 else 'a', index = False, header = (batch_ix == 0))

        b_count += 1
        if max_batches is not None and b_count >= max_batches:
            break

    return {'sample_df': pd.concat(sample_dfs), 'topk_df': pd.concat(topk_dfs), 'all_pre_mlp_hidden_states': all_pre_mlp_hidden_states}

res = run_and_export_topk(model, model_prefix, 'fw', test_dl, max_batches = None)

## Analyze here - file size too large

In [ ]:
"""
Let's clean up the mappings here. We'll get everything to a sample_ix level first.
"""

sample_df_raw =\
    res['sample_df']\
    .assign(sample_ix = lambda df: df.groupby(['batch_ix', 'sequence_ix', 'token_ix']).ngroup())\
    .assign(seq_id = lambda df: df.groupby(['batch_ix', 'sequence_ix']).ngroup())\
    .reset_index()

topk_df =\
    res['topk_df']\
    .merge(sample_df_raw[['sample_ix', 'batch_ix', 'sequence_ix', 'token_ix']], how = 'inner', on = ['sequence_ix', 'token_ix', 'batch_ix'])\
    .drop(columns = ['sequence_ix', 'token_ix', 'batch_ix'])

sample_df =\
    sample_df_raw\
    .drop(columns = ['batch_ix', 'sequence_ix'])

display(topk_df)
display(sample_df)

In [ ]:
"""
We need something to map our hidden state back onto the sample_df_raw batch_ix, sequence_ix, token_ix so that we can skip masked tokens

Let's get all unique sequence_ix, batch_ix in sample_df_raw (assumes all sequences have at least one non-masked token)
Then, we'll join onto all possible token_ix (assumes all token_ix have been represented at least one in sample_df_raw)
"""
merge_indices =\
    sample_df_raw[['batch_ix', 'sequence_ix']]\
    .drop_duplicates()\
    .merge(pd.DataFrame({'token_ix': sorted(sample_df_raw['token_ix'].drop_duplicates().tolist())}), how = 'cross')\
    .sort_values(by = ['batch_ix', 'sequence_ix', 'token_ix'])\
    .assign(original_index = lambda df: df.index)\
    .merge(
        sample_df_raw[['sample_ix', 'batch_ix', 'sequence_ix', 'token_ix']].assign(is_valid = 1),
        how = 'left',
        on = ['batch_ix', 'sequence_ix', 'token_ix']
    )\
    .pipe(lambda df: df[df['is_valid'] == 1])\
    .reset_index()

merge_indices

In [ ]:
"""
Now we can get the hidden states by layer, removing all the attention masked indices
"""
hs_by_layer = {}
for layer_ix in range(0, len(res['all_pre_mlp_hidden_states'][0])):
    hs_by_layer[layer_ix] = torch.cat(
        [batch_res[layer_ix] for batch_res in res['all_pre_mlp_hidden_states']], # A list of length num_batches, each of size BN x D
        dim = 0
    ) # Creates a list of n_all_tokens x D
    hs_by_layer[layer_ix] = hs_by_layer[layer_ix][merge_indices['original_index'].tolist(), :]

hs_by_layer[0].shape

In [ ]:
ufdg

In [ ]:
"""
Let's do raw k-means clustering
""" 
from sklearn.cluster import KMeans, MiniBatchKMeans

def cluster_kmeans(hs_for_layer: torch.Tensor):
    """
    Params:
        @hs_for_layer: A n_token_samples x D tensor

    Returns:
        A list of length n_token_samples of cluster ids
    """
    hs_layer = hs_for_layer.to(torch.float32)
    hs_layer_np = hs_layer
    kmeans = MiniBatchKMeans(n_clusters = 64, max_iter = 10000, batch_size = 2048 * 10, random_state = 123)
    kmeans.fit(hs_layer_np)
    cluster_labels = kmeans.labels_          # shape = (n_samples,)
    # cluster_centers = kmeans.cluster_centers_ # shape = (num_clusters, D)

    return cluster_labels.tolist()

kmeans_res = [{'cluster_method': 'kmeans', 'layer_ix': k, 'cluster_ids': cluster_kmeans(v)} for k, v in tqdm(hs_by_layer.items())]

In [ ]:
kmeans_res

In [ ]:
# """
# Some quick and dirty cross-layer tests
# """ 
# pd.set_option('display.max_rows', 100)
# joined_df =\
#     pd.concat([pd.DataFrame({'layer_' + str(x['layer_ix']) + '_id': x['cluster_ids']}) for x in kmeans_res], axis = 1)\
#     .assign(sample_ix = merge_indices['sample_ix'].tolist())\
#     .merge(sample_df, on = 'sample_ix', how = 'inner')

# joined_df\
#     .pipe(lambda df: df[(df['layer_0_id'] == 1) & (df['layer_1_id'] == 59) & (df['layer_2_id'] == 23) & (df['layer_3_id'] == 5) ])\
#     .head(100)

In [ ]:
sample_df

## Compare to Dense Models